In [25]:
import numpy as np
import cv2
import math
import sys, time

In [26]:
aa = np.array([[[1, 1, 1], [0, 1, 0]],[[1, 2, 3], [4, 5, 6]]]).astype(np.float32)
bb = aa.reshape(-1, 2, 3)
print(bb)
print("*****")
b = cv2.resize(bb, (0, 0), fx=2, fy=2, interpolation=cv2.INTER_CUBIC)
print(b)

[[[1. 1. 1.]
  [0. 1. 0.]]

 [[1. 2. 3.]
  [4. 5. 6.]]]
*****
[[[ 1.1499634   0.9279022   0.9390259 ]
  [ 0.67785645  0.82284546  0.46691895]
  [-0.09973145  0.6498108  -0.31066895]
  [-0.5718384   0.544754   -0.7827759 ]]

 [[ 1.0098877   1.1548767   1.4630127 ]
  [ 0.97875977  1.3805542   1.4318848 ]
  [ 0.92749023  1.7522583   1.3806152 ]
  [ 0.8963623   1.9779358   1.3494873 ]]

 [[ 0.7791748   1.528717    2.3260498 ]
  [ 1.4743652   2.2991333   3.0212402 ]
  [ 2.6193848   3.5680542   4.16626   ]
  [ 3.3145752   4.3384705   4.86145   ]]

 [[ 0.6390991   1.7556915   2.8500366 ]
  [ 1.7752686   2.856842    3.986206  ]
  [ 3.6466064   4.6705017   5.857544  ]
  [ 4.782776    5.771652    6.9937134 ]]]


In [84]:
# import modules
import cv2
import numpy as np
import math
import sys
import time
  
  
# Interpolation kernel

def u(s, a):
    if (abs(s) >= 0) & (abs(s) <= 1):
        return (a+2)*(abs(s)**3)-(a+3)*(abs(s)**2)+1
    elif (abs(s) > 1) & (abs(s) <= 2):
        return a*(abs(s)**3)-(5*a)*(abs(s)**2)+(8*a)*abs(s)-4*a
    return 0

  
# Padding
def padding(img, H, W, C):
    zimg = np.zeros((H+4, W+4, C))
    zimg[2:H+2, 2:W+2, :C] = img
      
    # Pad the first/last two col and row
    zimg[2:H+2, 0:2, :C] = img[:, 0:1, :C]
    zimg[H+2:H+4, 2:W+2, :] = img[H-1:H, :, :]
    zimg[2:H+2, W+2:W+4, :] = img[:, W-1:W, :]
    zimg[0:2, 2:W+2, :C] = img[0:1, :, :C]
      
    # Pad the missing eight points
    zimg[0:2, 0:2, :C] = img[0, 0, :C]
    zimg[H+2:H+4, 0:2, :C] = img[H-1, 0, :C]
    zimg[H+2:H+4, W+2:W+4, :C] = img[H-1, W-1, :C]
    zimg[0:2, W+2:W+4, :C] = img[0, W-1, :C]
    #print("---------------------"+"zimg"+"---------------------")
    #print(zimg)
    return zimg
  
  
# Bicubic operation
def bicubic(img, ratio, a):
    
    # Get image size
    H, W, C = img.shape
      
    # Here H = Height, W = weight,
    # C = Number of channels if the 
    # image is coloured.
    img = padding(img, H, W, C)
      
    # Create new image
    dH = math.floor(H*ratio)
    dW = math.floor(W*ratio)
  
    # Converting into matrix
    dst = np.zeros((dH, dW, C))  
    # np.zeroes generates a matrix 
    # consisting only of zeroes
    # Here we initialize our answer 
    # (dst) as zero
  
    h = 1/ratio
  
    print('Start bicubic interpolation')
    print('It will take a little while...')
    inc = 0
      
    for c in range(C):
        # set initial x and y from dst coordinate(0, 0) to src coordinate
        init_x = -0.5 + (0.5 / ratio) + 2
        init_y = -0.5 + (0.5 / ratio) + 2
        x = init_x
        y = init_y
        #count = 1
        for j in range(dH):
            x = init_x
            if(j != 0):
                y +=  h
            for i in range(dW):
                
                # Getting the coordinates of the
                # nearby values
                if( i != 0):
                    x +=  h
                
                x1 = math.floor(x) -1 - x
                x2 = math.floor(x) - x
                x3 = math.floor(x) + 1 - x
                x4 = math.floor(x) + 2 - x
  
                y1 = math.floor(y) -1 -y
                y2 = math.floor(y) -y
                y3 = math.floor(y) + 1 - y
                y4 = math.floor(y) + 2 - y
                  
                # Considering all nearby 16 values
                mat_l = np.matrix([[u(x1, a), u(x2, a), u(x3, a), u(x4, a)]])
                mat_m = np.matrix([[img[int(y+y1), int(x+x1), c],
                                    img[int(y+y2), int(x+x1), c],
                                    img[int(y+y3), int(x+x1), c],
                                    img[int(y+y4), int(x+x1), c]],
                                   [img[int(y+y1), int(x+x2), c],
                                    img[int(y+y2), int(x+x2), c],
                                    img[int(y+y3), int(x+x2), c],
                                    img[int(y+y4), int(x+x2), c]],
                                   [img[int(y+y1), int(x+x3), c],
                                    img[int(y+y2), int(x+x3), c],
                                    img[int(y+y3), int(x+x3), c],
                                    img[int(y+y4), int(x+x3), c]],
                                   [img[int(y+y1), int(x+x4), c],
                                    img[int(y+y2), int(x+x4), c],
                                    img[int(y+y3), int(x+x4), c],
                                    img[int(y+y4), int(x+x4), c]]])
                mat_r = np.matrix(
                    [[u(y1, a)], [u(y2, a)], [u(y3, a)], [u(y4, a)]])
                  
                # Here the dot function is used to get 
                # the dot product of 2 matrices
                dst[j, i, c] = np.dot(np.dot(mat_l, mat_m), mat_r)
                
                #print("x =",x)
                #print("y = ",y)
                #print(count)
                #count = count + 1
    # If there is an error message, it
    # directly goes to stderr
    sys.stderr.write('\n')
      
    # Flushing the buffer
    sys.stderr.flush()
    return dst

In [85]:
img = aa.reshape(-1, 2, 3)
print(img)
  
# Scale factor
ratio = 2
  
# Coefficient
a = -0.75
  
# Passing the input image in the 
# bicubic function
dst = bicubic(img, ratio, a)  
print(dst)

[[[1. 1. 1.]
  [0. 1. 0.]]

 [[1. 2. 3.]
  [4. 5. 6.]]]
Start bicubic interpolation
It will take a little while...
[[[ 1.14996338  0.92790222  0.93902588]
  [ 0.67785645  0.82284546  0.46691895]
  [-0.09973145  0.64981079 -0.31066895]
  [-0.57183838  0.54475403 -0.78277588]]

 [[ 1.0098877   1.15487671  1.4630127 ]
  [ 0.97875977  1.3805542   1.43188477]
  [ 0.92749023  1.7522583   1.38061523]
  [ 0.8963623   1.97793579  1.3494873 ]]

 [[ 0.7791748   1.52871704  2.3260498 ]
  [ 1.47436523  2.2991333   3.02124023]
  [ 2.61938477  3.5680542   4.16625977]
  [ 3.3145752   4.33847046  4.8614502 ]]

 [[ 0.63909912  1.75569153  2.85003662]
  [ 1.77526855  2.85684204  3.98620605]
  [ 3.64660645  4.67050171  5.85754395]
  [ 4.78277588  5.77165222  6.99371338]]]
